# Tratamento dos Dados

In [1]:
import pandas as pd
from unidecode import unidecode

In [34]:
# url = 'https://bi.s3.es.gov.br/covid19/MICRODADOS.zip'
url = '../MICRODADOS.zip'
df = pd.read_csv(url, sep = ';', encoding = 'cp1252')
df.head()

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido
0,2021-06-18,2021-06-15,2021-06-14,NaN,2021-06-18,NaN,NaN,2021-06-18,NaN,Descartados,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
1,2021-06-18,2021-06-16,2021-06-13,2021-06-18,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
2,2021-06-18,2021-06-17,2021-06-16,NaN,2021-06-17,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno
3,2021-06-18,2021-06-18,2021-06-15,NaN,2021-06-18,NaN,NaN,2021-06-18,NaN,Descartados,...,Não Informado,Não Informado,Ignorado,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
4,2021-06-18,2021-06-18,2021-06-14,NaN,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado


In [36]:
df['DataNotificacao'].min()

'2020-01-23'

In [3]:
df.query('Municipio == "SERRA" & Bairro == "COLINA DE LARANJEIRAS" & DataColetaTesteRapido == "2021-06-17"')

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido
3485,2021-06-17,2021-06-17,2021-06-15,NaN,2021-06-17,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
4103,2021-06-17,2021-06-17,2021-06-13,NaN,2021-06-17,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
4263,2021-06-17,2021-06-17,2021-06-14,NaN,2021-06-17,NaN,NaN,NaN,NaN,Descartados,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
5566,2021-06-17,2021-06-17,2021-06-13,NaN,2021-06-17,NaN,NaN,NaN,NaN,Confirmados,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno
7499,2021-06-17,2021-06-17,2021-06-07,NaN,2021-06-17,NaN,NaN,NaN,NaN,Descartados,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
7705,2021-06-17,2021-06-17,2021-06-17,NaN,2021-06-17,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno


In [4]:
df

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido
0,2021-06-18,2021-06-15,2021-06-14,NaN,2021-06-18,NaN,NaN,2021-06-18,NaN,Descartados,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
1,2021-06-18,2021-06-16,2021-06-13,2021-06-18,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
2,2021-06-18,2021-06-17,2021-06-16,NaN,2021-06-17,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno
3,2021-06-18,2021-06-18,2021-06-15,NaN,2021-06-18,NaN,NaN,2021-06-18,NaN,Descartados,...,Não Informado,Não Informado,Ignorado,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
4,2021-06-18,2021-06-18,2021-06-14,NaN,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567419,2020-02-27,2020-04-16,2020-02-16,2020-02-27,NaN,NaN,NaN,2020-04-16,NaN,Descartados,...,Não,Não,Não,Não Informado,Não Informado,Negativo,Não Informado,Não Informado,Não Informado,Não Informado
1567420,2020-02-27,2020-04-16,2020-02-15,2020-02-27,NaN,NaN,NaN,2020-04-16,NaN,Descartados,...,Não,Não,Não,Não Informado,Não Informado,Negativo,Não Informado,Não Informado,Não Informado,Não Informado
1567421,2020-02-27,2020-04-16,2020-02-15,2020-02-27,NaN,NaN,NaN,2020-04-16,NaN,Descartados,...,Não,Não,Não,Não Informado,Não Informado,Negativo,Não Informado,Não Informado,Não Informado,Não Informado
1567422,2020-02-25,2020-04-16,2020-02-13,2020-02-26,NaN,NaN,NaN,2020-04-16,2020-04-16,Descartados,...,Não,Sim,Não,Não,Não,Negativo,Não Informado,Não Informado,Não Informado,Não Informado


In [5]:
df.query('Evolucao == "Óbito pelo COVID-19"').shape

(11264, 45)

In [6]:
print('Classificacão: {}'.format(df['Classificacao'].unique()))
print('Evolução: {}'.format(df['Evolucao'].unique()))

Classificacão: ['Descartados' 'Suspeito' 'Confirmados']
Evolução: ['Ignorado' '-' 'Cura' 'Óbito pelo COVID-19' 'Óbito por outras causas']


In [7]:
total_casos = df.query('Classificacao == "Confirmados"').shape[0]
total_obitos = df.query('Evolucao == "Óbito pelo COVID-19"').shape[0]
total_curas = df.query('Evolucao == "Cura"').shape[0]

print('TOTAIS')
print('=================')
print('  - CASOS: {}'.format(total_casos))
print('  - ÓBITOS: {} ({}%)'.format(total_obitos, round((total_obitos * 100) / total_casos, 1)))
print('  - CURA: {} ({}%)'.format(total_curas, round((total_curas * 100) / total_casos, 1)))

TOTAIS
  - CASOS: 505764
  - ÓBITOS: 11264 (2.2%)
  - CURA: 482970 (95.5%)


In [8]:
df['DataConfirmado'] = df['DataColeta_RT_PCR'].combine_first(df['DataColetaTesteRapido'])\
    .combine_first(df['DataColetaSorologia'])\
    .combine_first(df['DataColetaSorologiaIGG'])

df

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido,DataConfirmado
0,2021-06-18,2021-06-15,2021-06-14,NaN,2021-06-18,NaN,NaN,2021-06-18,NaN,Descartados,...,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno,2021-06-18
1,2021-06-18,2021-06-16,2021-06-13,2021-06-18,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,2021-06-18
2,2021-06-18,2021-06-17,2021-06-16,NaN,2021-06-17,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,2021-06-17
3,2021-06-18,2021-06-18,2021-06-15,NaN,2021-06-18,NaN,NaN,2021-06-18,NaN,Descartados,...,Não Informado,Ignorado,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno,2021-06-18
4,2021-06-18,2021-06-18,2021-06-14,NaN,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567419,2020-02-27,2020-04-16,2020-02-16,2020-02-27,NaN,NaN,NaN,2020-04-16,NaN,Descartados,...,Não,Não,Não Informado,Não Informado,Negativo,Não Informado,Não Informado,Não Informado,Não Informado,2020-02-27
1567420,2020-02-27,2020-04-16,2020-02-15,2020-02-27,NaN,NaN,NaN,2020-04-16,NaN,Descartados,...,Não,Não,Não Informado,Não Informado,Negativo,Não Informado,Não Informado,Não Informado,Não Informado,2020-02-27
1567421,2020-02-27,2020-04-16,2020-02-15,2020-02-27,NaN,NaN,NaN,2020-04-16,NaN,Descartados,...,Não,Não,Não Informado,Não Informado,Negativo,Não Informado,Não Informado,Não Informado,Não Informado,2020-02-27
1567422,2020-02-25,2020-04-16,2020-02-13,2020-02-26,NaN,NaN,NaN,2020-04-16,2020-04-16,Descartados,...,Sim,Não,Não,Não,Negativo,Não Informado,Não Informado,Não Informado,Não Informado,2020-02-26


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567424 entries, 0 to 1567423
Data columns (total 46 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   DataNotificacao          1567424 non-null  object
 1   DataCadastro             1567424 non-null  object
 2   DataDiagnostico          1567424 non-null  object
 3   DataColeta_RT_PCR        774977 non-null   object
 4   DataColetaTesteRapido    550045 non-null   object
 5   DataColetaSorologia      86282 non-null    object
 6   DataColetaSorologiaIGG   132939 non-null   object
 7   DataEncerramento         1343980 non-null  object
 8   DataObito                14387 non-null    object
 9   Classificacao            1567424 non-null  object
 10  Evolucao                 1567424 non-null  object
 11  CriterioConfirmacao      1567424 non-null  object
 12  StatusNotificacao        1567424 non-null  object
 13  Municipio                1567424 non-null  object
 14  Ba

In [10]:
df.query('DataConfirmado == "2021-06-15"')

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido,DataConfirmado
210,2021-06-18,2021-06-18,2021-06-15,2021-06-15,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Ignorado,Não,Não,Negativo,Não Informado,Não Informado,Não Informado,Não Informado,2021-06-15
292,2021-06-18,2021-06-18,2021-06-14,NaN,2021-06-15,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno,2021-06-15
414,2021-06-18,2021-06-18,2021-06-15,NaN,NaN,2021-06-15,2021-06-15,NaN,NaN,Confirmados,...,Não Informado,Não,Não,Não,Não Informado,Não Informado,Reagente,Reagente,Não Informado,2021-06-15
444,2021-06-18,2021-06-18,2021-06-15,NaN,2021-06-15,NaN,NaN,NaN,NaN,Descartados,...,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno,2021-06-15
498,2021-06-18,2021-06-18,2021-06-12,2021-06-15,NaN,NaN,NaN,NaN,NaN,Confirmados,...,Não Informado,Não,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,2021-06-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221177,2021-05-03,2021-05-03,2021-05-01,NaN,2021-06-15,NaN,NaN,2021-06-18,NaN,Descartados,...,Não,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno,2021-06-15
275386,2021-04-20,2021-04-21,2021-04-17,NaN,2021-06-15,NaN,NaN,2021-06-15,NaN,Suspeito,...,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno,2021-06-15
390961,2021-03-28,2021-03-28,2021-03-25,2021-06-15,2021-03-28,NaN,NaN,2021-04-16,NaN,Descartados,...,Não,Não,Sim,Não,Positivo,Negativo,Não Informado,Não Informado,Teste rápido Antígeno,2021-06-15
403258,2021-03-25,2021-03-29,2021-03-09,NaN,2021-06-15,2021-03-24,2021-03-24,2021-04-03,NaN,Confirmados,...,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Reagente,Reagente,Teste rápido Antígeno,2021-06-15


In [11]:
df.dropna(subset=['DataConfirmado'], inplace=True)

In [26]:
df['DataNotificacao'] = pd.to_datetime(df['DataConfirmado'])
df['DataObito'] = pd.to_datetime(df['DataObito'])
df.sort_values('DataNotificacao', inplace=True)
df = df.query('Classificacao == "Confirmados"').reset_index(drop=True)

## Padroniza nome de municípios e bairros

In [14]:
df['Municipio'] = df['Municipio'].apply(lambda x: unidecode(str(x)).upper())
df['Bairro'] = df['Bairro'].apply(lambda x: unidecode(str(x)).upper())

## Cálculo de Confirmados, Óbitos e Curados

In [15]:
grupo_base = ['DataNotificacao', 'Municipio', 'Bairro']

datas = df[['DataNotificacao']].drop_duplicates().reset_index(drop=True)
municipios_bairros = df[['Municipio', 'Bairro']].drop_duplicates().reset_index(drop=True)
    
datas['key'] = 0
municipios_bairros['key'] = 0

grupo_base

['DataNotificacao', 'Municipio', 'Bairro']

In [16]:
df['Confirmados'] = 1
df['Curas'] = df['Evolucao'].apply(lambda e: 1 if e == 'Cura' else 0)
df['Obitos'] = df['Evolucao'].apply(lambda e: 1 if e == 'Óbito pelo COVID-19' else 0)

df_confirmados_curas = df[['Municipio', 'Bairro', 'DataNotificacao', 'Confirmados', 'Curas']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .sum()\
    .reset_index()

df_obitos = df[['Municipio', 'Bairro', 'DataObito', 'Obitos']]\
    .groupby(['Municipio', 'Bairro', 'DataObito'])\
    .sum()\
    .reset_index()\
    .rename({'DataObito': 'DataNotificacao'}, axis=1)

df_counts = df_confirmados_curas.merge(
    df_obitos,
    on=grupo_base,
    how='outer'
).fillna(0)

In [17]:
df_datas_municipios = pd.merge(datas, municipios_bairros, how='outer')[grupo_base]

df_counts = df_datas_municipios.merge(
    df_counts,
    on=grupo_base,
    how='left'
)

df_counts.fillna(
    {
        'Confirmados': 0,
        'Obitos': 0,
        'Curas': 0,
    },
    inplace=True
)

In [18]:
df_counts.query('Municipio == "SERRA" & Bairro == "COLINA DE LARANJEIRAS"').tail(6)

,DataNotificacao,Municipio,Bairro,Confirmados,Curas,Obitos
2680496,2021-06-14,SERRA,COLINA DE LARANJEIRAS,3.0,0.0,0.0
2685563,2021-06-15,SERRA,COLINA DE LARANJEIRAS,4.0,1.0,0.0
2690630,2021-06-16,SERRA,COLINA DE LARANJEIRAS,3.0,0.0,0.0
2695697,2021-06-17,SERRA,COLINA DE LARANJEIRAS,1.0,0.0,0.0
2700764,2021-06-18,SERRA,COLINA DE LARANJEIRAS,0.0,0.0,0.0
2705831,2089-12-27,SERRA,COLINA DE LARANJEIRAS,0.0,0.0,0.0


In [19]:
columns_sum = ['Confirmados', 'Obitos', 'Curas']
df_counts_by_week = df_counts.groupby(['Municipio', 'Bairro', pd.Grouper(key='DataNotificacao', freq='W', label='left', closed='left')])[columns_sum]\
    .sum()\
    .reset_index()\
    .sort_values('DataNotificacao')

df_counts_by_week['ConfirmadosAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Confirmados']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week['ObitosAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Obitos']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week['CurasAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Curas']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week.query('Municipio == "SERRA" & Bairro == "COLINA DE LARANJEIRAS"').tail(6)

,Municipio,Bairro,DataNotificacao,Confirmados,Obitos,Curas,ConfirmadosAcumulado,ObitosAcumulado,CurasAcumulado
412372,SERRA,COLINA DE LARANJEIRAS,2021-05-16,40.0,0.0,39.0,2640.0,26.0,2611.0
412373,SERRA,COLINA DE LARANJEIRAS,2021-05-23,31.0,0.0,31.0,2671.0,26.0,2642.0
412374,SERRA,COLINA DE LARANJEIRAS,2021-05-30,26.0,0.0,26.0,2697.0,26.0,2668.0
412375,SERRA,COLINA DE LARANJEIRAS,2021-06-06,16.0,1.0,14.0,2713.0,27.0,2682.0
412376,SERRA,COLINA DE LARANJEIRAS,2021-06-13,13.0,0.0,1.0,2726.0,27.0,2683.0
412377,SERRA,COLINA DE LARANJEIRAS,2089-12-25,0.0,0.0,0.0,2726.0,27.0,2683.0


In [31]:
today = date.today().strftime('%Y-%m-%d')
df_counts_by_week.query(f'DataNotificacao <= "{today}"')

,Municipio,Bairro,DataNotificacao,Confirmados,Obitos,Curas,ConfirmadosAcumulado,ObitosAcumulado,CurasAcumulado
0,ABADIA DOS DOURADOS,NAN,1920-12-26,0.0,0.0,0.0,0.0,0.0,0.0
81686,BREJETUBA,RANCHO DA ANTA,1920-12-26,0.0,0.0,0.0,0.0,0.0,0.0
392826,SAO MATEUS,ARNALDO BASTOS,1920-12-26,0.0,0.0,0.0,0.0,0.0,0.0
81780,BREJETUBA,SANTA RITA,1920-12-26,0.0,0.0,0.0,0.0,0.0,0.0
392732,SAO MATEUS,ALVORADA,1920-12-26,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
143818,CONCEICAO DO CASTELO,FAZENDA BOA VISTA,2021-06-13,0.0,0.0,0.0,7.0,1.0,6.0
137238,COLATINA,SAO MIGUEL,2021-06-13,0.0,0.0,0.0,174.0,2.0,169.0
336142,PORTO AMAZONAS,NAN,2021-06-13,0.0,0.0,0.0,2.0,0.0,0.0
474792,VITORIA,SAO JOSE,2021-06-13,1.0,0.0,1.0,435.0,11.0,423.0


In [20]:
df_counts_by_week.query('Municipio == "SERRA" & Bairro == "COLINA DE LARANJEIRAS"').tail(6)

,Municipio,Bairro,DataNotificacao,Confirmados,Obitos,Curas,ConfirmadosAcumulado,ObitosAcumulado,CurasAcumulado
412372,SERRA,COLINA DE LARANJEIRAS,2021-05-16,40.0,0.0,39.0,2640.0,26.0,2611.0
412373,SERRA,COLINA DE LARANJEIRAS,2021-05-23,31.0,0.0,31.0,2671.0,26.0,2642.0
412374,SERRA,COLINA DE LARANJEIRAS,2021-05-30,26.0,0.0,26.0,2697.0,26.0,2668.0
412375,SERRA,COLINA DE LARANJEIRAS,2021-06-06,16.0,1.0,14.0,2713.0,27.0,2682.0
412376,SERRA,COLINA DE LARANJEIRAS,2021-06-13,13.0,0.0,1.0,2726.0,27.0,2683.0
412377,SERRA,COLINA DE LARANJEIRAS,2089-12-25,0.0,0.0,0.0,2726.0,27.0,2683.0


In [21]:
df_counts_by_week_new = df_counts_by_week.rename({'DataNotificacao': 'DataInicio'}, axis=1)

In [22]:
from datetime import date, timedelta

def get_data_interval(data_inicio):
    data_fim = data_inicio + timedelta(days=6)
    today = date.today()
    
    if today < data_fim:
        data_fim = today
        
    pattern = '%d/%m/%Y'

    return f'{data_inicio.strftime(pattern)} - {data_fim.strftime(pattern)}'

df_counts_by_week_new['DataInicio'].apply(get_data_interval).tolist()

['26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01/1921',
 '26/12/1920 - 01/01

In [23]:
df_counts_by_week_new.tail()

,Municipio,Bairro,DataInicio,Confirmados,Obitos,Curas,ConfirmadosAcumulado,ObitosAcumulado,CurasAcumulado
361523,SANTA LEOPOLDINA,REGENCIA,2089-12-25,0.0,0.0,0.0,11.0,0.0,11.0
266489,MARATAIZES,JACARANDA,2089-12-25,0.0,0.0,0.0,385.0,4.0,372.0
205389,IRUPI,CORREGO BARRA DE SANTA ROSA,2089-12-25,0.0,0.0,0.0,17.0,0.0,15.0
112423,CARIACICA,VILA ROMA,2089-12-25,0.0,0.0,0.0,15.0,0.0,14.0
476297,ZE DOCA,NAN,2089-12-25,0.0,0.0,0.0,1.0,0.0,1.0


In [24]:
df_counts_by_week['Obitos'].sum()

11094.0

In [25]:
# df_counts_by_week.to_csv('microdados_pre-processed.csv', sep=',', index=False, encoding='UTF-8')